In [1]:
print("hello world")

hello world


In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

# -------------------------------
# 1. Data Cleaning
# -------------------------------
# Load CSV
df = pd.read_csv("D:\Guvi_projects\Luxury_house\Luxury_Housing_Bangalore.csv")

print("Original Shape:", df.shape)

# Remove duplicates
df = df.drop_duplicates()

# Handle missing values
df = df.dropna(subset=["price"])  # Drop rows where price is missing
df["location"] = df["location"].fillna("Unknown")  # Fill missing locations
df["size"] = df["size"].fillna(df["size"].mode()[0])  # Fill with most common size

# Fix data types
if "price" in df.columns:
    df["price"] = pd.to_numeric(df["price"], errors="coerce")

if "bath" in df.columns:
    df["bath"] = pd.to_numeric(df["bath"], errors="coerce")

# Replace invalid/negative values
num_cols = ["price", "bath"]
for col in num_cols:
    df[col] = np.where(df[col] < 0, np.nan, df[col])
    df[col] = df[col].fillna(df[col].median())

print("\n✅ After Cleaning Shape:", df.shape)

# -------------------------------
# 2. Data Preprocessing & Transformation
# -------------------------------
# Feature Engineering: Extract number of bedrooms from "size" column if it exists
if "size" in df.columns:
    df["bhk"] = df["size"].str.extract(r"(\d+)").astype(float)

# Normalize/scale prices (log transform to handle skewness)
df["price_log"] = np.log1p(df["price"])

# Example transformation: categorize price ranges
df["price_category"] = pd.cut(
    df["price"],
    bins=[0, 50, 100, 200, np.inf],
    labels=["Budget", "Mid-range", "Premium", "Luxury"]
)

print("\n✅ Preprocessing Done")
print(df.head())

# -------------------------------
# 3. SQL Data Warehousing & Querying
# -------------------------------
# Create SQLAlchemy engine
engine = create_engine("sqlite:///luxury_housing_cleaned.db")

# Store cleaned + transformed data
df.to_sql("luxury_housing", con=engine, if_exists="replace", index=False)

# Example SQL Queries
with engine.connect() as conn:
    # Top 5 most expensive houses
    result = conn.execute(text("""
        SELECT location, price 
        FROM luxury_housing 
        ORDER BY price DESC 
        LIMIT 5
    """)).fetchall()
    print("\n🏠 Top 5 Most Expensive Houses:")
    for row in result:
        print(row)

    # Average price per location
    avg_prices = conn.execute(text("""
        SELECT location, AVG(price) as avg_price
        FROM luxury_housing
        GROUP BY location
        ORDER BY avg_price DESC
        LIMIT 5
    """)).fetchall()
    print("\n📍 Top 5 Locations by Avg Price:")
    for row in avg_prices:
        print(row)

    # Count of houses per price category
    categories = conn.execute(text("""
        SELECT price_category, COUNT(*) as total
        FROM luxury_housing
        GROUP BY price_category
    """)).fetchall()
    print("\n💰 House Counts by Price Category:")
    for row in categories:
        print(row)


<>:9: SyntaxWarning: invalid escape sequence '\G'
<>:9: SyntaxWarning: invalid escape sequence '\G'
C:\Users\VENKATARAMAN\AppData\Local\Temp\ipykernel_12876\1240877807.py:9: SyntaxWarning: invalid escape sequence '\G'
  df = pd.read_csv("D:\Guvi_projects\Luxury_house\Luxury_Housing_Bangalore.csv")


Original Shape: (101000, 18)


KeyError: ['price']